# Dependences

In [1]:
include("../../src/struct_data.jl")

save_data

# Model

In [2]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_4_MiniTest",
        path_output = ""
    ) 
)

dump(model)

  0.000003 seconds (5 allocations: 208 bytes)


ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 100000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_4_MiniTest"
    path_output: String ""


In [3]:
# Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.06)
Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.12)
agg_size = 15

15

# Aggregate

In [4]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.12)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 

In [5]:
# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType(
#         "HEK_1", 
#         InteractionPar(Par1, Par2),
#         Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
#     )], 
#     model
# )

# kNN

In [6]:
include("../../src/neighbor.jl")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

display(agg.Simulation.Neighbor.idx_red)

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  2461  2462  2463  2468  2469  2470
  6   7   8  49   1   2   3   3  58     2502  2503  2504  2498  2499  2500
  5   6   7  50  50   1   2  53  15     2503  2504  2455  2499  2500  2501
 42  43  44   5  51  51  52  54  10     2454  2455  2456  2461  2462  2463
 43  44  45  10   6  52  53   7  49     2453  2454  2500  2460  2461  2462
  2   3   2   9   4   7   8  14   4  …  2498  2499  2495  2503  2502  2503
 35   1  37  40  11   5   6  13  66     2500  2501  2496  2469  2504  2471
 37  36  39  42  10  12  13  44  50     2493  2494  2462  2467  2468  2469
 51  38  53  58  41  11  12  46  48     2494  2495  2464  2453  2470  2455
  0  52   0   0  43  42  43  62   0     2460  2461  2447     0  2454     0
  ⋮                   ⋮              ⋱                 ⋮              
  0   0   0   0   0   0   0   0   0        0     0     0     0     0     0
  0   0   0   0   0   0   0   0   0        0     0  

# Sum Forces

In [7]:
include("../../src/forces/forces.jl")

threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Force.N_i,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        pi/4,
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)

  0.712121 seconds (1.10 M CPU allocations: 

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  7.13207f-12  -1.49224f-5    0.000364654
  0.0888629     0.0638758    -0.0488428
  7.13207f-12  -1.49224f-5    0.000364654
  0.0511999     0.0691472     0.0841245
 -8.91938f-6    0.000149172   0.000364654
  0.0190545     0.105342     -0.0541754
 -4.6549f-12    0.000164602   0.000364654
  0.00568408    0.0897711    -0.0792325
 -0.0469086    -0.210187      0.106451
  7.43841f-12  -0.000158756   0.000364828
  ⋮                          
 -3.38513f-5    9.98547f-5   -0.000364828
 -0.0658291     0.0586792    -0.0816516
 -8.91939f-6   -0.000149172  -0.000364654
  0.108221      0.0494119     0.0148142
 -4.6549f-12   -0.000164602  -0.000364654
 -0.226155      0.0712024     0.0364979
  0.105627      0.0556566     0.0117545
  7.13207f-12   1.49224f-5   -0.000364654
 -7.13207f-12   1.49224f-5   -0.000364654

71.889 MiB, 3.02% gc time)


# MiniTest

In [8]:
include("../../src/run_event.jl")

run_test(agg, model, "mini Testing")
display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

mini Testing...   1%|▎                                   |  ETA: 0:00:14

mini Testing...   6%|██▏                                 |  ETA: 0:00:21

mini Testing...   7%|██▍                                 |  ETA: 0:00:20

mini Testing...   7%|██▌                                 |  ETA: 0:00:21

mini Testing...  10%|███▌                                |  ETA: 0:00:19

...

mini Testing... 100%|████████████████████████████████████| Time: 0:00:20


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.00450541   -0.00779202   -0.0128236
 -0.00244016   -0.00868553   -0.00349094
  0.0768874    -0.0992956     0.0265485
  0.022103     -0.0076704    -0.234569
 -0.0724413    -0.0734495    -0.0596485
  0.00529977    0.0124755     0.00240333
  0.00711772   -0.00318719   -0.00973332
 -0.169402     -0.184681      0.0126662
 -0.000828427  -0.00724518    0.00523606
 -0.00454155    0.00736318    0.00641771
  ⋮                          
 -0.00268407    0.00989513    0.010222
 -0.00176133    0.00595436    0.0151148
  0.00787016    0.00547755    0.112678
  0.00344317   -0.000568822   0.00438036
  0.00623379   -0.00773627    0.0103905
  0.00495708   -0.0110539     0.00692341
  0.00807525   -0.00018339    0.0159176
  0.00531789   -0.00522861   -0.0986903
  0.0416438    -0.0271247    -0.0979624

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -5.42885   -3.95298   -10.8245
 -2.06694   -6.75211   -10.0897
  1.13329   -3.13068   -11.6737
 -6.48616    2.64711    -8.70461
 -1.46616    1.57822   -12.8991
 -1.09535   -4.26488    -9.00377
 -0.04057   -1.94904   -11.3558
 -1.43784   -2.5504    -11.6221
 -2.46889    2.1472    -11.346
 -5.37039   -1.011     -10.2992
  ⋮                    
  5.35978    0.320962   11.0407
 -6.68367    4.16824     8.96516
 -1.87149    1.33888    10.3181
  1.87407    3.01515    11.4886
  6.4533     4.12187     8.35027
  6.20955    3.2993      9.65895
 -0.984512   3.47173    10.8704
  0.462783   4.27907    11.1077
  2.27953    4.70238    11.1735